# Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import LabelEncoder, PolynomialFeatures, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from scipy.stats import ttest_rel
from imblearn.over_sampling import SMOTE

# Load the dataset

In [2]:
df = pd.read_csv('/home/eric_baldwin/ddiMain/capstone/ddi_capstone_2/data/space_decay.csv')
print(df.head())

   CCSDS_OMM_VERS                            COMMENT        CREATION_DATE  \
0               2  GENERATED VIA SPACE-TRACK.ORG API  2021-11-01T06:46:11   
1               2  GENERATED VIA SPACE-TRACK.ORG API  2021-11-01T04:58:37   
2               2  GENERATED VIA SPACE-TRACK.ORG API  2021-11-01T06:26:11   
3               2  GENERATED VIA SPACE-TRACK.ORG API  2021-10-31T18:07:15   
4               2  GENERATED VIA SPACE-TRACK.ORG API  2021-11-01T04:58:37   

  ORIGINATOR      OBJECT_NAME   OBJECT_ID CENTER_NAME REF_FRAME TIME_SYSTEM  \
0    18 SPCS  ARIANE 42P+ DEB   1992-072J       EARTH      TEME         UTC   
1    18 SPCS         SL-8 DEB   1979-028C       EARTH      TEME         UTC   
2    18 SPCS           GSAT 1   2001-015A       EARTH      TEME         UTC   
3    18 SPCS         CZ-4 DEB  1999-057MB       EARTH      TEME         UTC   
4    18 SPCS         CZ-4 DEB  1999-057MC       EARTH      TEME         UTC   

  MEAN_ELEMENT_THEORY  ... RCS_SIZE  COUNTRY_CODE  LAUNCH_DATE

# Drop unnecessary columns

In [3]:
df = df.drop(labels=['DECAY_DATE'], axis=1)
print(df.head())

   CCSDS_OMM_VERS                            COMMENT        CREATION_DATE  \
0               2  GENERATED VIA SPACE-TRACK.ORG API  2021-11-01T06:46:11   
1               2  GENERATED VIA SPACE-TRACK.ORG API  2021-11-01T04:58:37   
2               2  GENERATED VIA SPACE-TRACK.ORG API  2021-11-01T06:26:11   
3               2  GENERATED VIA SPACE-TRACK.ORG API  2021-10-31T18:07:15   
4               2  GENERATED VIA SPACE-TRACK.ORG API  2021-11-01T04:58:37   

  ORIGINATOR      OBJECT_NAME   OBJECT_ID CENTER_NAME REF_FRAME TIME_SYSTEM  \
0    18 SPCS  ARIANE 42P+ DEB   1992-072J       EARTH      TEME         UTC   
1    18 SPCS         SL-8 DEB   1979-028C       EARTH      TEME         UTC   
2    18 SPCS           GSAT 1   2001-015A       EARTH      TEME         UTC   
3    18 SPCS         CZ-4 DEB  1999-057MB       EARTH      TEME         UTC   
4    18 SPCS         CZ-4 DEB  1999-057MC       EARTH      TEME         UTC   

  MEAN_ELEMENT_THEORY  ... OBJECT_TYPE  RCS_SIZE  COUNTRY_CODE

# Rename object type for better presentation

In [4]:
df['OBJECT_TYPE'] = df['OBJECT_TYPE'].replace({'DEBRIS': 'Debris', 'PAYLOAD': 'Payload', 'TBA': 'Unknown', 'ROCKET BODY': 'Rocket'})
print(df['OBJECT_TYPE'].unique())


['Debris' 'Payload' 'Rocket' 'Unknown']


# Fill missing values for categorical columns with 'Unknown'

In [10]:
df['COUNTRY_CODE'] = df['COUNTRY_CODE'].replace(to_replace={'TBD': 'Unknown', np.nan: 'Unknown'})
df['RCS_SIZE'] = df['RCS_SIZE'].replace(to_replace={np.nan: 'Unknown'})
print(df['OBJECT_TYPE'].head())

0     Debris
1     Debris
2    Payload
3     Debris
4     Debris
Name: OBJECT_TYPE, dtype: object


# Create PERIOD_HOURS and ALTITUDE_MI columns

In [7]:
df['PERIOD_HOURS'] = df['PERIOD'] / 60
df['ALTITUDE_MI'] = (df['SEMIMAJOR_AXIS'] - 6371) * 0.6213
print(df[['PERIOD_HOURS', 'ALTITUDE_MI']].head())

   PERIOD_HOURS   ALTITUDE_MI
0      8.214400   8883.110063
1      1.744817    613.246709
2     23.116400  21637.923148
3      1.624267    400.164419
4      1.629933    410.306520


# Convert OBJECT_TYPE to binary classification

In [11]:
df['OBJECT_TYPE'] = df['OBJECT_TYPE'].apply(lambda x: 1 if x == 'Payload' else 0)
print(df['OBJECT_TYPE'].value_counts())

0    9422
1    4950
Name: OBJECT_TYPE, dtype: int64


# Encode categorical variables

In [12]:
le_country_code = LabelEncoder()
df['COUNTRY_CODE'] = le_country_code.fit_transform(df['COUNTRY_CODE'])
le_rcs_size = LabelEncoder()
df['RCS_SIZE'] = le_rcs_size.fit_transform(df['RCS_SIZE'])
print(df[['COUNTRY_CODE', 'RCS_SIZE']].head())

   COUNTRY_CODE  RCS_SIZE
0            29         1
1            18         2
2            36         0
3            67         2
4            67         2


# Fill missing values for numerical columns

In [13]:
imputer = SimpleImputer(strategy='mean')
numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns
df[numerical_cols] = imputer.fit_transform(df[numerical_cols])
print(df[numerical_cols].isna().sum())

CCSDS_OMM_VERS       0
MEAN_MOTION          0
ECCENTRICITY         0
INCLINATION          0
RA_OF_ASC_NODE       0
ARG_OF_PERICENTER    0
MEAN_ANOMALY         0
EPHEMERIS_TYPE       0
NORAD_CAT_ID         0
ELEMENT_SET_NO       0
REV_AT_EPOCH         0
BSTAR                0
MEAN_MOTION_DOT      0
MEAN_MOTION_DDOT     0
SEMIMAJOR_AXIS       0
PERIOD               0
APOAPSIS             0
PERIAPSIS            0
OBJECT_TYPE          0
RCS_SIZE             0
COUNTRY_CODE         0
LAUNCH_DATE          0
FILE                 0
GP_ID                0
PERIOD_HOURS         0
ALTITUDE_MI          0
dtype: int64


# Feature selection

In [14]:
features = ['INCLINATION', 'PERIOD_HOURS', 'ALTITUDE_MI', 'ECCENTRICITY', 'RA_OF_ASC_NODE', 'ARG_OF_PERICENTER', 'MEAN_ANOMALY', 'SEMIMAJOR_AXIS', 'APOAPSIS', 'PERIAPSIS']
X = df[features]
y = df['OBJECT_TYPE']
print(X.head())

   INCLINATION  PERIOD_HOURS   ALTITUDE_MI  ECCENTRICITY  RA_OF_ASC_NODE  \
0       7.7156      8.214400   8883.110063      0.652893         90.2410   
1      82.9193      1.744817    613.246709      0.003072        299.1120   
2      12.1717     23.116400  21637.923148      0.023739         16.5368   
3      98.4781      1.624267    400.164419      0.006062          8.7205   
4      98.4232      1.629933    410.306520      0.006226        122.0724   

   ARG_OF_PERICENTER  MEAN_ANOMALY  SEMIMAJOR_AXIS   APOAPSIS  PERIAPSIS  
0           243.1216       38.7796       20668.618  27784.871    796.095  
1           158.9093      201.3337        7358.038   1002.507    957.299  
2           250.1248      146.2900       41197.852  35797.696  33841.738  
3            37.3771      323.1632        7015.076    679.465    594.417  
4           345.1605       27.6061        7031.400    697.039    609.491  


# Standardize features

In [15]:
scaler = StandardScaler()
X = scaler.fit_transform(X)
print(X[:5])

[[-2.24934758  0.60322852  1.04719468  3.22640749 -0.78921453  0.73851912
  -1.38560132  1.04719468  1.69514994 -0.25166856]
 [ 0.2891097  -0.26614145 -0.34217606 -0.35306489  1.00038759 -0.06766244
   0.09391195 -0.34217606 -0.36252374 -0.23137287]
 [-2.09893451  2.60573016  3.19005524 -0.23922497 -1.42071049  0.80556218
  -0.40707747  3.19005524  2.31077069  3.90880033]
 [ 0.81428797 -0.28234072 -0.37797475 -0.33659587 -1.48768037 -1.23111503
   1.20276348 -0.37797475 -0.38734287 -0.27705996]
 [ 0.81243486 -0.28157925 -0.37627084 -0.33569414 -0.5164838   1.71535836
  -1.48729879 -0.37627084 -0.38599267 -0.27516213]]


# Handle class imbalance using SMOTE

In [16]:
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)
print("Original class distribution:", np.bincount(y))
print("Resampled class distribution:", np.bincount(y_resampled))

Original class distribution: [9422 4950]
Resampled class distribution: [9422 9422]


# Split data into training and test sets

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42)
print("Training set size:", X_train.shape)
print("Test set size:", X_test.shape)

Training set size: (13190, 10)
Test set size: (5654, 10)


# Define Logistic Regression model & cross-validation to get accuracy scores for Logistic Regression without polynomial features

In [18]:
logreg = LogisticRegression(max_iter=1000, class_weight='balanced')

logreg_scores = cross_val_score(logreg, X_train, y_train, cv=5, scoring='accuracy')
print("Logistic Regression scores without polynomial features:", logreg_scores)

Defined Logistic Regression model.
Logistic Regression scores without polynomial features: [0.70887036 0.71721001 0.71683093 0.70887036 0.70470053]


# Added polynomial features & split data into training and test sets for polynomial features

In [21]:

poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X)
X_poly_resampled, y_poly_resampled = smote.fit_resample(X_poly, y)

X_poly_train, X_poly_test, y_poly_train, y_poly_test = train_test_split(X_poly_resampled, y_poly_resampled, test_size=0.3, random_state=42)
print("Training set size with polynomial features:", X_poly_train.shape)
print("Test set size with polynomial features:", X_poly_test.shape)

Training set size with polynomial features: (13190, 65)
Test set size with polynomial features: (5654, 65)


# Cross-validation to get accuracy scores for Logistic Regression with polynomial features

In [22]:
logreg_poly_scores = cross_val_score(logreg, X_poly_train, y_poly_train, cv=5, scoring='accuracy')
print("Logistic Regression scores with polynomial features:", logreg_poly_scores)

Logistic Regression scores with polynomial features: [0.8146323  0.80022745 0.8085671  0.79378317 0.78809704]


# Perform paired t-test

In [23]:
t_stat, p_value = ttest_rel(logreg_scores, logreg_poly_scores)
print(f"t-statistic: {t_stat}, p-value: {p_value}")

t-statistic: -20.8863499474633, p-value: 3.105225148998835e-05


# Results

In [24]:
alpha = 0.05
if p_value < alpha:
    print("Reject the null hypothesis - there is a significant difference between the two configurations")
else:
    print("Fail to reject the null hypothesis - no significant difference between the two configurations")

Reject the null hypothesis - there is a significant difference between the two configurations
